# League of Legends Web Scraper 4.0

This juypter notebook contains the script that I used to generate my League of Legends match history data set using the Riot Games API library (Cassiopeia).

In [ ]:
# Imports
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
import csv
import pandas as pd
import numpy as np

## Setup
In order to use the data mining script I created for League of Legends matches, you must have an official Riot API key which can be found here: https://developer.riotgames.com/

The Riot Games API library (Cassiopeia) is a public library which can be used to scrap data from League of Legends matches.

Replace the Riot API Key below with a new key (expires daily 24hr).

In [ ]:
# Replace expired key below
key = "RGAPI-f60b8ba8-c124-4505-90b8-373dc49752a2"
cass.set_riot_api_key(key)  

## Populating the Data Set

My methodology for populating my match history data set is by first generating a "seed" which which will influence what type of players that will populate the data set.

The idea behind my methodology is to first create a data set which contains matches that the "seed" player played in, and then apply the webscraping script again for each player that is already in the data set. 

The reasoning behind this is that the Riot Games API only allows basic members to retrieve the last 20 match history from a single player, and this isn't enough data to make any insightful models.

In [ ]:
# Insert summoner name here to populate the csv file.
seed = "lzd"

summoner = cass.get_summoner(name=seed, region="NA")
print("{name} is a level {level} summoner on the {region} server.".format(name=summoner.name, level=summoner.level, region=summoner.region))

match_history = cass.get_match_history(continent=summoner.region.continent, puuid=summoner.puuid, queue=Queue.ranked_solo_fives)
file_name = "match_history_test.csv"

with open(file_name, "a", newline="", encoding='utf-8') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=",", quotechar="|", quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

    for match in match_history:
        for p in match.participants:
            player_info = []
            player_info += [match.id, match.duration, p.summoner.name, p.side.name, p.team_position.name, p.summoner.level, p.stats.win, p.champion.name, p.stats.kills, p.stats.deaths, p.stats.assists, p.stats.damage_dealt_to_turrets, p.stats.gold_earned, p.stats.turret_takedowns, p.stats.vision_score, p.stats.total_minions_killed, p.stats.first_blood_kill, p.stats.first_tower_kill]
            filewriter.writerow(player_info)

The cell below generates the last 20 match history for each player which played in the games with the "seed" player.

Despite the limitation for the amount of match history data we can collect, this is a creative way to collect match history data of a similar population of players to our "seed" player.

There will be duplications in our data set, and should be fixed in the exploratory data analysis portion of any future projects.

In [ ]:
data = pd.read_csv("match_history.csv")
data = data.drop_duplicates()
player = np.unique(data["summoner_name"].tolist()[::-1])


for name_ in player:
    try:
        if name_ != seed:
            summoner = cass.get_summoner(name=name_, region="NA")
            
            print("{name} is a level {level} summoner on the {region} server.".format(name=summoner.name, level=summoner.level, region=summoner.region))
            
            match_history = cass.get_match_history(continent=summoner.region.continent, puuid=summoner.puuid, queue=Queue.ranked_solo_fives)
            file_name = "match_history_test.csv"
            
            with open(file_name, "a", newline="", encoding='utf-8') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=",", quotechar="|", quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

                for match in match_history:
                    for p in match.participants:
                        player_info = []
                        player_info += [match.id, match.duration, p.summoner.name, p.side.name, p.team_position.name, p.summoner.level, p.stats.win, p.champion.name, p.stats.kills, p.stats.deaths, p.stats.assists, p.stats.damage_dealt_to_turrets, p.stats.gold_earned, p.stats.turret_takedowns, p.stats.vision_score, p.stats.total_minions_killed, p.stats.first_blood_kill, p.stats.first_tower_kill]
                        filewriter.writerow(player_info)
    except:
        continue